In [ ]:
# ##  3.1 预判是否为identical-subtitle
# def predict_same(BtimeFrameList):
#     BtimeIndenticalList = []
#     BtimeIndenticalList.append(True)  # 第一个一定是独立的
#     for i in range(len(BtimeFrameList)-1):
#         firstGray = cv.cvtColor(BtimeFrameList[i][100:135,5:,:],cv.COLOR_RGB2GRAY)
#         secondGray = cv.cvtColor(BtimeFrameList[i+1][100:135,5:,:],cv.COLOR_RGB2GRAY)
#         diff = np.sum(np.power((firstGray - secondGray),2))/np.size(firstGray)
#         if diff > 90:
#             BtimeIndenticalList.append(True)  # 发生了变化就是独立的
#         else:
#             BtimeIndenticalList.append(False)  #没发生变化则不是独立的
#     return BtimeIndenticalList

# BtimeIndenticalList = predict_same(BtimeFrameList)  # True表示当前帧是发生了变化的
#print(len(BtimeFrameList))
#firstGray = cv.cvtColor(BtimeFrameList[9][100:135,5:,:],cv.COLOR_RGB2GRAY)
#secondGray = cv.cvtColor(BtimeFrameList[10][100:135,5:,:],cv.COLOR_RGB2GRAY)
#diff = np.sum(np.power((firstGray - secondGray),2))/(np.size(firstGray))
#print(BtimeIndenticalList)
#plt.imshow(BtimeFrameList[25][100:135,5:,:])

In [ ]:
# ##  3.2 判断是否为字幕样式
# BtimeFrame_preStt_List = []
# for i in range(len(BtimeFrameList)):
#     if BtimeIndenticalList[i]:
#         BtimeFrame_preStt_List.append(predictStt(BtimeFrameList[i]))  # 是独立帧也得是预字幕帧
#     else:
#         BtimeFrame_preStt_List.append(False)  # 不是独立帧就不要了
        
# #  最终筛选出来可以用于检测字幕的帧的索引
# subtitle_frameIndex = [i for i,x in enumerate(BtimeFrame_preStt_List) if x==True]
#print(subtitle_frameIndex)  
#ttt = BtimeFrameList[228]
#plt.imshow(ttt)
#plt.imshow(cv.cvtColor(ttt[100:135,5:,:],cv.COLOR_RGB2GRAY))
#print(cv.cvtColor(ttt[100:135,5:,:],cv.COLOR_RGB2GRAY))

In [4]:
import cv2 as cv
import numpy as np
import os
import math
from aip import AipOcr
import json
import glob
import matplotlib.pyplot as plt

In [5]:
def get_OCRClient():
    APP_ID = '17843970'
    API_KEY = 'LcYOkBpgw41RVYDarm49wyN9'
    SECRET_KEY = 'eRclWcYSuO7IoZCIvbyEiBuCpzoOdy3x'
    
    client = AipOcr(APP_ID, API_KEY, SECRET_KEY)
    return client

def extractWord(client, frameList, frameIsSubtitle):
    #  传入client, 用于找字幕的系列图片和对应的预测其是否为标题
    subFrame_path = 'subFrame/'
    subTitle_path = 'subTitle/'
    p_format = '.jpg'
    message_list = []
    for i in range(len(frameList)):
        # 如果是subTitle
        if(frameIsSubtitle[i]):
            img = cv.imread(subFrame_path+str(frameList[i])+p_format)
            cv.imwrite(subTitle_path+str(frameList[i])+p_format,img[400:,130:,:])
            i = open(subTitle_path+str(frameList[i])+p_format,'rb')
            message_list.append(client.basicGeneral(i.read()))
        # plt.imwrite(frameList[subtitleIndex[i]],'pic/'+str(subtitleIndex[i])+'.jpg')
    return message_list

In [6]:
BGframeList = np.int64(np.loadtxt('subFrame/BGFrameIndex.txt'))
BGframeIsSubtitle = np.int64(np.loadtxt('subFrame/BGFrameIsSubtitle.txt'))

In [7]:
ocrClient = get_OCRClient()  # 获得api_client

In [8]:
messageList = extractWord(ocrClient, BGframeList, BGframeIsSubtitle)  # 提取字幕中的字

In [14]:
print(messageList)

[{'log_id': 1040280379794873885, 'words_result_num': 0, 'words_result': []}, {'log_id': 4604078089535004029, 'words_result_num': 0, 'words_result': []}, {'log_id': 3175411112244881949, 'words_result_num': 3, 'words_result': [{'words': '霞'}, {'words': '康辉'}, {'words': '央电视台「新闻联播201814'}]}, {'log_id': 6533209163005776797, 'words_result_num': 0, 'words_result': []}, {'log_id': 3610983099519390813, 'words_result_num': 2, 'words_result': [{'words': '习近平在视察中部战区陆军某师时强调'}, {'words': '大抓实战化军事训练聚力打造精锐作战力量'}]}, {'log_id': 8319807427377947165, 'words_result_num': 0, 'words_result': []}, {'log_id': 8379978110009506557, 'words_result_num': 0, 'words_result': []}, {'log_id': 6180630703623867101, 'words_result_num': 0, 'words_result': []}, {'log_id': 8552104093632330205, 'words_result_num': 0, 'words_result': []}, {'log_id': 768285361667528413, 'words_result_num': 0, 'words_result': []}, {'log_id': 692666618522072701, 'words_result_num': 0, 'words_result': []}, {'log_id': 1227480756259332829, 'words_r

In [13]:
#  将 1_获取的字幕 和 2_第一次出现此字幕的帧的index 存下来
outfile = 'result_title.txt'
dict_list = []
mge_count = 0
for i in range(len(BGframeList)):
    if(BGframeIsSubtitle[i]):
        count = 0
        str_temp = ''
        for text in messageList[mge_count].get('words_result'):
            if count !=0:
                str_temp += ', '
            str_temp += text.get('words')
            count += 1
        print(str_temp)
        mge_count += 1
        dict_list.append({
            'bg': float(BGframeList[i]),
            'title': str(str_temp)
        })
        

    
with open(outfile, 'a+', encoding='utf-8') as fo:
    fo.writelines(json.dumps(dict_list,ensure_ascii=False))  # 要存储中文的必须操作，第二个参数
    fo.writelines('\n')




霞, 康辉, 央电视台「新闻联播201814

习近平在视察中部战区陆军某师时强调, 大抓实战化军事训练聚力打造精锐作战力量











i



习近平在视察中部战区陆军某师时强调, 大抓实战化军事训练聚力打造精锐作战力量








1

















































习近平


央视记者刘元锋刘朋朋杨照飞刘笑宇张麒张振李骏




习近平
鸟



牢记主席训令练兵备战谋打嬴

陆军第82集团军某合成旅侦察科长董建国, 实现强军目标是奋斗出来的
陆军第82集团军某合成旅侦察科长董建国, 面对难题逢山开路遇水架桥
陆军第82集团军某合成旅侦察科长董建国, 让部队的战斗力得到真正锤炼
陆军第82集团军某合成旅侦察科长董建国, 以踏石留印抓铁有痕的韧动
陆军第82集团军某合成旅侦察科长董建国, 不断提升部队战斗力水平
海军排冈山舰舰长杨扬, 2018年
海军井冈山舰舰长杨扬, 牢记习主席的关怀和嘱托
海军井冈山舰舰长杨扬
海军井冈山舰舰长杨扬, 切实提高我们的打赢能力


中部战区陆军某师副师长朱廷超, 聆听主席训令后, ,
中部战区陆军某师副师长朱廷超, 打响了年度开训的第一枪
中部战区陆军某师副师长朱廷超, 下步我们坚决贯彻主席训会, ∥
中部战区陆军某师副师长朱廷超, 突出指挥信息系统应用训练
中部战区陆军某师副师长朱廷超, 指挥谋略训练和官买战斗技能
中部战区陆军某师副师长朱廷超, 基于网络信息系统的体系作战能力
空军某飞行训练基地团长陈启强, 开16苏30敏10同步开始组训
空军某飞行训练基地团长陈启强
空军莱飞行训练基地团长除启强, 我们感受到这种大抓实战化训练
空军某飞行训练基地团长陈启强, 者说这种训练的热情
空军某飞机练地团长除启强6, 感染了我们每一个官兵
果除3, 我自己感觉军人生来就是为打仗的

中部战区联合训练局副局长张付样, 始终突出联战联训这个重心
中部战反联答练局副长张付样, 始终突出联战联训这个重心
中部战区联合训练局副局长张付样, 设联合系统
中部战区联合训练局副局长张付样, 向科技要战斗力
陆军第7集团军某合成旅营长度东阳, 我你侦察兵由传统侦条
陆军第73集团军某合成旅营长廖东阳, 新年度训练
围报收整分报段影训
陆军第73集团军某合成旅营长廖

In [ ]:
#  测试存储的文件是否能打开
tfile = open(outfile, 'rb')  # 打开一个json文件
tcontext = json.load(tfile)  # 传一个文件，读出内容转换成自定
tcontext[0]['subtitle']